In [18]:
from collections import defaultdict
from unidecode import unidecode
from fuzzywuzzy import fuzz
from nameparser import HumanName
import pandas as pd
import urllib.request
from urllib.error import HTTPError
import xml.etree.ElementTree as ET

In [25]:
class URLs:
    FINANCE = 'https://cg-519a459a-0ea3-42c2-b7bc-fa1143481f74.s3-us-gov-west-1.amazonaws.com/bulk-downloads/2018/candidate_summary_2018.csv'
    FORECAST = 'https://projects.fivethirtyeight.com/congress-model-2018/house_district_forecast.csv'
    NICKNAMES = 'http://deron.meranda.us/data/nicknames.txt'
    

In [26]:
def finance_fetch():
    """Returns a file-like object of the candidates Excel file"""
    return urllib.request.urlopen(URLs.FINANCE)

def forecast_fetch():
    """Returns a file-like object of the 538 House Forecast csv"""
    return urllib.request.urlopen(URLs.FORECAST)

def nicknames_fetch():
    """Returns a file-like object of the Nicknames csv"""
    return urllib.request.urlopen(URLs.NICKNAMES)

In [31]:
def forecast_read():
    forecast_file = forecast_fetch()
    forecast = pd.read_csv(forecast_file)
    latest_date = forecast['forecastdate'].max()
    latest_forecast = forecast[(forecast['forecastdate'] == latest_date)]
    classic = latest_forecast[(latest_forecast['model'] == 'classic')]
    return classic

def nicknames_read():
    nicknames_file = nicknames_fetch()
    nicksnames_csv = pd.read_csv(nicknames_file, comment='#', sep="\s+", header=None, names=["nickname", "firstname", "prob"], index_col=False)
    nicks = defaultdict(dict)
    for index, row in nicksnames_csv.iterrows():
        nicks[row["nickname"]][row["firstname"]] = row["prob"]
    return nicks

def finance_read():
    finance_file = finance_fetch()
    finance_csv = pd.read_csv(finance_file)
    house = finance_csv[(finance_csv['Cand_Office'] == 'H')]
    return house

In [120]:
def forecast_clean(forecast):
    forecast['district_code'] = forecast['state'] + forecast['district'].astype(str).str.zfill(2)
    forecast['name'] = forecast['candidate'].map(lambda c: HumanName(unidecode(c).upper()))
    forecast = forecast.sort_values(['district_code', 'win_probability'], ascending=[True, False])
    forecast = forecast.groupby(['district_code']).head(n=2)
    return forecast

def finance_clean(finance):
    finance['name'] = finance['Cand_Name'].map(lambda c: HumanName(unidecode(c).upper()))
    finance['party'] = finance['Cand_Party_Affiliation'].astype(str).apply(lambda x: x[0:1])
    # SEC data has district = 0 for single-district states
    finance.loc[ finance['Cand_Office_Dist'] == 0, 'Cand_Office_Dist' ] = 1
    finance['district_code'] = finance['Cand_Office_St'] + finance['Cand_Office_Dist'].astype(str).str.zfill(2)
    finance['candidate_id'] = finance['Cand_Id']
    finance['finance_total'] = finance['Total_Receipt']
    finance['finance_cash'] = finance['Cash_On_Hand_COP']
    return finance

In [7]:
# Utility functions

def mean(nums):
    return float(sum(nums)) / len(nums)

def mapnone(value, func):
    return None if value is None else func(value)

In [84]:
# Name matching
NAME_THRESHOLD = 85

def namedict(name):
    nam = name.upper()
    literal = {nam: 1.0}
    nicks = nicknames.get(nam, {})
    return {**literal, **nicks}

def firstname_distance(name1, name2):
    namedict1 = namedict(name1)
    namedict2 = namedict(name2)
    
    return max([
        fuzz.ratio(nick1, nick2) * namedict1[nick1] * namedict2[nick2]
        for nick1 in namedict1
        for nick2 in namedict2
    ])        

def name_distance(name1, name2):
    first = max([
        firstname_distance(name1.first, name2.first),
        firstname_distance(name1.first, name2.middle),
        firstname_distance(name1.middle, name2.first)        
    ])
    
    last = max([
        fuzz.ratio(name1.last, name2.last),
        fuzz.ratio(name1.last, name2.middle),
        fuzz.ratio(name1.middle, name2.last)
    ])
    
    return mean([
        first,
        last
    ])

def matches(target, candidates):
    # consider only those of the same district
    district_candidates = candidates[(candidates['district_code'] == target['district_code'])]
    distances = district_candidates.apply(lambda c: name_distance(c['name'], target['name']), axis=1)
    distances.sort_values(ascending=False)
    return distances

def best_match(forecast_candidate, candidates):
    """Returns matching candidate from SEC list, or None"""
    distances = matches(forecast_candidate, candidates)
        
    close_distances = distances[(distances > NAME_THRESHOLD)]
    if close_distances.size > 0:
        return candidates.loc[close_distances.index]['candidate_id']
    else:
        return None
    
def match_candidates(forecast, finance):
    forecast['candidate_id'] = forecast.apply(lambda r: mapnone(best_match(r, finance), lambda m: m.values[0]), axis=1)
    return forecast
    

In [160]:
def match_finance(forecast, finance):
    return forecast.merge(finance[['candidate_id', 'finance_total', 'finance_cash']], on=['candidate_id'], how='left')

def match_competitors(forecast):
    firstplace = forecast.groupby(['district_code']).nth(0)
    secondplace = forecast.groupby(['district_code']).nth(1)
    f1 = firstplace.merge(secondplace[['candidate_id', 'candidate', 'win_probability', 'finance_total', 'finance_cash']], left_index=True, right_index=True, how='left', suffixes=('', '_competitor'))
    f2 = secondplace.merge(firstplace[['candidate_id', 'candidate', 'win_probability', 'finance_total', 'finance_cash']], left_index=True, right_index=True, how='left', suffixes=('', '_competitor'))
    f = f1.append(f2)
    f['win_probability_margin'] = abs(f['win_probability'] - f['win_probability_competitor'])
    f['win_probability_margin_approx'] = f['win_probability_margin'].round(1)
    f['finance_total_ratio'] = f['finance_total'] / f['finance_total_competitor']
    f['finance_total_ratio_approx'] = f['finance_total_ratio'].round(1)
    f.set_index('candidate_id')
    return f

In [32]:
# Read data from sources
finance = finance_read()
forecast = forecast_read()
nicknames = nicknames_read()

In [137]:
# Basic clean
finance = finance_clean(finance)
f0 = forecast_clean(forecast)

In [161]:
# Match 538 to SEC data
f1 = match_candidates(f0, finance)
f2 = match_finance(f1, finance)
f3 = match_competitors(f2)

In [166]:
close = f3.sort_values(by=['win_probability_margin_approx', 'finance_total_ratio_approx'], ascending=[True, True])
close[close['party'] == 'D']

,candidate,candidate_id,district,finance_cash,finance_total,forecastdate,incumbent,model,name,p10_voteshare,...,win_probability,candidate_id_competitor,candidate_competitor,win_probability_competitor,finance_total_competitor,finance_cash_competitor,win_probability_margin,win_probability_margin_approx,finance_total_ratio,finance_total_ratio_approx
district_code,,,,,,,,,,,,,,,,,,,,,
FL26,Debbie Mucarsel-Powell,H8FL26039,26,686438.32,3276504.38,2018-10-20,False,classic,DEBBIE MUCARSEL-POWELL,45.56,...,0.5057,H4FL26038,Carlos Curbelo,0.4943,4463962.89,1570220.62,0.0114,0.0,0.733990,0.7
WA08,Kim Schrier,H8WA08189,8,1767344.47,5343212.64,2018-10-20,False,classic,KIM SCHRIER,45.50,...,0.5011,H8WA08205,Dino Rossi,0.4989,4098669.12,1094063.21,0.0022,0.0,1.303646,1.3
KY06,Amy McGrath,H8KY06164,6,1750828.36,6690748.56,2018-10-20,False,classic,AMY MCGRATH,44.55,...,0.5174,H0KY06104,Andy Barr,0.4826,4393263.91,1297754.15,0.0348,0.0,1.522956,1.5
MI08,Elissa Slotkin,H8MI08102,8,1833173.17,5487818.85,2018-10-20,False,classic,ELISSA SLOTKIN,44.13,...,0.5156,H4MI08135,Mike Bishop,0.4844,2953167.11,1277739.94,0.0312,0.0,1.858283,1.9
NC09,Dan McCready,H8NC09123,9,1648496.67,4328512.47,2018-10-20,False,classic,DAN MCCREADY,44.38,...,0.5069,H8NC09131,Mark Harris,0.4931,1618586.06,468701.58,0.0138,0.0,2.674255,2.7
VA05,Leslie Cockburn,H8VA05155,5,1029644.37,2435652.61,2018-10-20,False,classic,LESLIE COCKBURN,46.00,...,0.5177,H8VA05171,Denver Riggleman,0.4823,912564.57,505089.11,0.0354,0.0,2.669019,2.7
PA01,Scott Wallace,H8PA08174,1,1417372.94,9814245.81,2018-10-20,False,classic,SCOTT WALLACE,45.68,...,0.4944,H6PA08277,Brian Fitzpatrick,0.5056,2979264.91,1202025.52,0.0112,0.0,3.294184,3.3
TX07,Lizzie Pannill Fletcher,None,7,NaN,NaN,2018-10-20,False,classic,LIZZIE PANNILL FLETCHER,45.43,...,0.4806,H0TX07055,John Culberson,0.5194,2825090.10,1054247.84,0.0388,0.0,NaN,NaN
IL06,Sean Casten,H8IL06139,6,1392412.95,4731374.52,2018-10-20,False,classic,SEAN CASTEN,45.18,...,0.4480,H6IL06117,Peter J. Roskam,0.5520,5866626.63,1858259.87,0.1040,0.1,0.806490,0.8
